In [0]:
%sql
create database agg;

-- Switch to the appropriate database
USE DATABASE agg;

CREATE OR REPLACE STAGE my_stage2;

select * from silver_table;

-- Set the session parameter for timestamp input format
ALTER SESSION SET TIMESTAMP_INPUT_FORMAT = 'YYYY-MM-DD"T"HH24:MI:SS.FF3TZHTZM';

-- Define the file format with correct timestamp handling
CREATE OR REPLACE FILE FORMAT my_csv_format
  TYPE = 'CSV'
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  SKIP_HEADER = 1
  NULL_IF = ('NULL', 'null')
  DATE_FORMAT = 'YYYY-MM-DD'
  TIME_FORMAT = 'HH24:MI:SS.FF3TZHTZM';

-- Create or replace the stage
CREATE OR REPLACE STAGE my_stage2
  FILE_FORMAT = my_csv_format;

-- Create the table
CREATE OR REPLACE TABLE silver_table (
    Country_ID VARCHAR,
    Date DATE,
    Site_ID VARCHAR,
    Product_ID VARCHAR,
    Volume_Sold VARCHAR,
    load_timestamp TIMESTAMP_TZ,
    Volume_Sold_Liters DOUBLE,
    Global_Country_ID VARCHAR,
    Global_Product_ID VARCHAR,
    Global_Product_Category VARCHAR,
    Global_Site_ID VARCHAR,
    update_timestamp TIMESTAMP_TZ
);

-- List files in the stage (optional, for debugging)
LIST @my_stage2;

-- Load data from the stage into the table
COPY INTO silver_table
FROM @my_stage2/Silver_table_afterSCD1.csv
FILE_FORMAT = my_csv_format
ON_ERROR = 'CONTINUE';

-- Validate data loading and format errors
COPY INTO silver_table
FROM @my_stage2/Silver_table_afterSCD1.csv
FILE_FORMAT = my_csv_format
VALIDATION_MODE = 'RETURN_ERRORS';



select * from silver_table;



CREATE OR REPLACE TABLE gold_table (
    Country_ID STRING,
    Date DATE,
    Site_ID STRING,
    Product_ID STRING,
    Volume_Sold STRING,
    load_timestamp TIMESTAMP_TZ,
    Volume_Sold_Liters DOUBLE,
    Global_Country_ID STRING,
    Global_Product_ID STRING,
    Global_Product_Category STRING,
    Global_Site_ID STRING,
    Volume_Sold_Last_Week DOUBLE,
    Volume_Sold_Last_Month DOUBLE,
    Volume_Sold_Last_Year DOUBLE,
    Pct_Change_Last_Week DOUBLE,
    Pct_Change_Last_Month DOUBLE,
    Pct_Change_Last_Year DOUBLE,
    Rolling_7_Day_Sum DOUBLE,
    Pct_Change_Rolling_7_Day_Sum DOUBLE
);


INSERT INTO gold_table
SELECT
    s.Country_ID,
    s.Date,
    s.Site_ID,
    s.Product_ID,
    s.Volume_Sold,
    s.load_timestamp,
    s.Volume_Sold_Liters,
    s.Global_Country_ID,
    s.Global_Product_ID,
    s.Global_Product_Category,
    s.Global_Site_ID,
    COALESCE(lw.Volume_Sold_Liters, 0) AS Volume_Sold_Last_Week,
    COALESCE(lm.Volume_Sold_Liters, 0) AS Volume_Sold_Last_Month,
    COALESCE(ly.Volume_Sold_Liters, 0) AS Volume_Sold_Last_Year,
    CASE 
        WHEN lw.Volume_Sold_Liters IS NULL OR lw.Volume_Sold_Liters = 0 THEN NULL
        ELSE (s.Volume_Sold_Liters - lw.Volume_Sold_Liters) / lw.Volume_Sold_Liters * 100
    END AS Pct_Change_Last_Week,
    CASE 
        WHEN lm.Volume_Sold_Liters IS NULL OR lm.Volume_Sold_Liters = 0 THEN NULL
        ELSE (s.Volume_Sold_Liters - lm.Volume_Sold_Liters) / lm.Volume_Sold_Liters * 100
    END AS Pct_Change_Last_Month,
    CASE 
        WHEN ly.Volume_Sold_Liters IS NULL OR ly.Volume_Sold_Liters = 0 THEN NULL
        ELSE (s.Volume_Sold_Liters - ly.Volume_Sold_Liters) / ly.Volume_Sold_Liters * 100
    END AS Pct_Change_Last_Year,
    COALESCE(rs.Rolling_7_Day_Sum, 0) AS Rolling_7_Day_Sum,
    CASE 
        WHEN rs.Last_Rolling_Sum IS NULL OR rs.Last_Rolling_Sum = 0 THEN NULL
        ELSE (rs.Rolling_7_Day_Sum - rs.Last_Rolling_Sum) / rs.Last_Rolling_Sum * 100
    END AS Pct_Change_Rolling_7_Day_Sum
FROM
    silver_table s
LEFT JOIN 
    silver_table lw 
    ON s.Country_ID = lw.Country_ID 
    AND s.Site_ID = lw.Site_ID 
    AND s.Product_ID = lw.Product_ID 
    AND s.Date = DATEADD('day', 7, lw.Date)
LEFT JOIN 
    silver_table lm 
    ON s.Country_ID = lm.Country_ID 
    AND s.Site_ID = lm.Site_ID 
    AND s.Product_ID = lm.Product_ID 
    AND s.Date = DATEADD('day', 30, lm.Date)
LEFT JOIN 
    silver_table ly 
    ON s.Country_ID = ly.Country_ID 
    AND s.Site_ID = ly.Site_ID 
    AND s.Product_ID = ly.Product_ID 
    AND s.Date = DATEADD('year', 1, ly.Date)
LEFT JOIN (
    SELECT
        Country_ID,
        Site_ID,
        Product_ID,
        Date,
        SUM(Volume_Sold_Liters) AS Rolling_7_Day_Sum,
        LAG(SUM(Volume_Sold_Liters)) OVER (PARTITION BY Country_ID, Site_ID, Product_ID ORDER BY Date) AS Last_Rolling_Sum
    FROM
        (SELECT
            Country_ID,
            Site_ID,
            Product_ID,
            Date,
            SUM(Volume_Sold_Liters) OVER (PARTITION BY Country_ID, Site_ID, Product_ID ORDER BY Date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS Volume_Sold_Liters
         FROM
            silver_table
        ) AS inner_rs
    GROUP BY
        Country_ID,
        Site_ID,
        Product_ID,
        Date
) rs
ON s.Country_ID = rs.Country_ID 
AND s.Site_ID = rs.Site_ID 
AND s.Product_ID = rs.Product_ID 
AND s.Date = rs.Date;

select* from gold_table;

SELECT COUNT(*) AS row_count
FROM gold_table;

SELECT COUNT(*) AS row_count
FROM silver_table;